# Análise de Readmissão Hospitalar de Pacientes Diabéticos

## Projeto de Predição de Readmissão Hospitalar em Menos de 30 Dias

Este notebook implementa um pipeline completo de análise e pré-processamento de dados para predição de readmissão hospitalar de pacientes diabéticos em menos de 30 dias.

### Objetivos:
1. **Análise Exploratória**: Compreender a estrutura e qualidade dos dados
2. **Limpeza de Dados**: Remover registros inválidos e tratar dados faltantes
3. **Engenharia de Features**: Preparar dados para algoritmos de Machine Learning
4. **Preparação para Modelagem**: Dividir dados em conjuntos de treino e teste

### Dataset:
- **Fonte**: UCI Machine Learning Repository - Diabetes 130-US hospitals for years 1999-2008 Data Set
- **Registros**: 101,766 encontros hospitalares
- **Features**: 50 variáveis (demográficas, clínicas, medicamentos)
- **Target**: Readmissão em <30 dias, >30 dias, ou não readmitido

---

## 1. Importar Bibliotecas

Importando todas as bibliotecas necessárias para análise de dados, visualização e machine learning.

In [ ]:
# Bibliotecas para manipulação de dados
import pandas as pd
import numpy as np
import os
import sys

# Bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliotecas para machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Configurações para visualização
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Configurações do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Importar módulos customizados do projeto
sys.path.append('../src')
try:
    from config import *
    print("✅ Módulos do projeto importados com sucesso")
except ImportError:
    print("⚠️ Módulos do projeto não encontrados, usando caminhos relativos")
    
print("📚 Todas as bibliotecas importadas com sucesso!")

## 2. Análise Exploratória dos Dados Processados

Carregando o dataset original e realizando exploração inicial para compreender a estrutura dos dados.

In [ ]:
# Carregando o dataset original
data_path = '../data/diabetic_data.csv'
df_raw = pd.read_csv(data_path)

print("="*80)
print("INFORMAÇÕES BÁSICAS DO DATASET")
print("="*80)

print(f"\n📊 Tamanho do dataset: {df_raw.shape}")
print(f"📋 Número de colunas: {df_raw.shape[1]}")
print(f"📈 Número de registros: {df_raw.shape[0]}")
print(f"👥 Número de pacientes únicos: {df_raw['patient_nbr'].nunique()}")
print(f"🏥 Número de encontros hospitalares: {len(df_raw)}")

print(f"\n📋 Tipos de dados por coluna:")
print(df_raw.dtypes.value_counts())

print(f"\n🔍 Primeiras 5 linhas do dataset:")
display(df_raw.head())

print(f"\n📈 Estatísticas descritivas das colunas numéricas:")
display(df_raw.describe())

## 3. Avaliação da Qualidade dos Dados

Analisando a qualidade dos dados, identificando valores faltantes, dados inconsistentes e problemas que precisam ser tratados.

In [ ]:
print("="*80)
print("ANÁLISE DE QUALIDADE DOS DADOS")
print("="*80)

# 1. Verificar valores nulos tradicionais
print("\n🔍 Análise de valores nulos (NaN):")
null_counts = df_raw.isnull().sum()
if null_counts.sum() > 0:
    null_summary = pd.DataFrame({
        'Coluna': null_counts.index,
        'Valores_Nulos': null_counts.values,
        'Percentual': (null_counts.values / len(df_raw)) * 100
    })
    null_summary = null_summary[null_summary['Valores_Nulos'] > 0].sort_values('Valores_Nulos', ascending=False)
    display(null_summary)
else:
    print("✅ Nenhum valor nulo (NaN) encontrado")

# 2. Verificar valores '?' (representam dados faltantes neste dataset)
print("\n🔍 Análise de valores faltantes ('?'):")
missing_summary = []
for col in df_raw.columns:
    if df_raw[col].dtype == 'object':
        missing_count = (df_raw[col] == '?').sum()
        if missing_count > 0:
            missing_pct = (missing_count / len(df_raw)) * 100
            missing_summary.append({
                'Coluna': col,
                'Valores_Faltantes': missing_count,
                'Percentual': f"{missing_pct:.1f}%"
            })

if missing_summary:
    missing_df = pd.DataFrame(missing_summary)
    missing_df = missing_df.sort_values('Valores_Faltantes', ascending=False)
    display(missing_df)
    
    # Visualizar colunas com mais dados faltantes
    plt.figure(figsize=(12, 6))
    top_missing = missing_df.head(10)
    plt.barh(top_missing['Coluna'], top_missing['Valores_Faltantes'])
    plt.title('Top 10 Colunas com Mais Dados Faltantes')
    plt.xlabel('Número de Valores Faltantes')
    plt.tight_layout()
    plt.show()
else:
    print("✅ Nenhum valor '?' encontrado")

# 3. Verificar distribuição das colunas categóricas principais
print("\n🔍 Distribuição das principais colunas categóricas:")
categorical_cols = ['race', 'gender', 'age', 'medical_specialty', 'readmitted']

for col in categorical_cols:
    if col in df_raw.columns:
        print(f"\n📋 {col}:")
        value_counts = df_raw[col].value_counts()
        print(value_counts.head(10))
        if '?' in value_counts.index:
            missing_pct = (value_counts['?'] / len(df_raw)) * 100
            print(f"   ❗ Dados faltantes ('?'): {value_counts['?']} ({missing_pct:.1f}%)")

# 4. Verificar duplicatas
print(f"\n🔍 Análise de duplicatas:")
print(f"Registros totais: {len(df_raw)}")
print(f"Registros únicos por encounter_id: {df_raw['encounter_id'].nunique()}")
print(f"Pacientes únicos: {df_raw['patient_nbr'].nunique()}")
print(f"Pacientes com múltiplas internações: {len(df_raw) - df_raw['patient_nbr'].nunique()}")

## 4. Análise e Engenharia da Variável Alvo

Analisando a variável alvo 'readmitted' e criando uma variável binária para predição de readmissão em menos de 30 dias.

In [ ]:
print("="*80)
print("ANÁLISE DA VARIÁVEL ALVO")
print("="*80)

# Analisar distribuição original da variável readmitted
print("\n🎯 Distribuição da variável 'readmitted':")
readmitted_counts = df_raw['readmitted'].value_counts()
readmitted_pct = df_raw['readmitted'].value_counts(normalize=True) * 100

readmitted_summary = pd.DataFrame({
    'Categoria': readmitted_counts.index,
    'Contagem': readmitted_counts.values,
    'Percentual': readmitted_pct.values.round(2)
})
display(readmitted_summary)

# Visualizar distribuição
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de barras
readmitted_counts.plot(kind='bar', ax=ax1, color=['skyblue', 'lightcoral', 'lightgreen'])
ax1.set_title('Distribuição da Variável Readmitted')
ax1.set_xlabel('Categoria de Readmissão')
ax1.set_ylabel('Número de Pacientes')
ax1.tick_params(axis='x', rotation=45)

# Gráfico de pizza
ax2.pie(readmitted_counts.values, labels=readmitted_counts.index, autopct='%1.1f%%', 
        colors=['skyblue', 'lightcoral', 'lightgreen'])
ax2.set_title('Distribuição Percentual da Readmissão')

plt.tight_layout()
plt.show()

# Criar variável alvo binária
print("\n🔧 Criando variável alvo binária:")
print("Definição: readmitted == '<30' → 1 (readmitido em <30 dias)")
print("           readmitted == '>30' ou 'NO' → 0 (não readmitido em <30 dias)")

df_raw['target'] = df_raw['readmitted'].apply(lambda x: 1 if x == '<30' else 0)

print(f"\n📊 Distribuição da variável alvo binária:")
target_counts = df_raw['target'].value_counts()
target_pct = df_raw['target'].value_counts(normalize=True) * 100

target_summary = pd.DataFrame({
    'Classe': ['Não Readmitido <30 dias', 'Readmitido <30 dias'],
    'Contagem': [target_counts[0], target_counts[1]],
    'Percentual': [target_pct[0].round(2), target_pct[1].round(2)]
})
display(target_summary)

print(f"\n📈 Métricas da variável alvo:")
print(f"Proporção de readmissões em <30 dias: {df_raw['target'].mean():.3f}")
print(f"Razão de desbalanceamento: {target_counts[0] / target_counts[1]:.1f}:1")

# Verificar se há valores nulos na variável alvo
null_readmitted = df_raw['readmitted'].isnull().sum()
null_target = df_raw['target'].isnull().sum()
print(f"\n🔍 Valores nulos:")
print(f"Valores nulos em 'readmitted': {null_readmitted}")
print(f"Valores nulos em 'target': {null_target}")

if null_readmitted == 0 and null_target == 0:
    print("✅ Nenhum valor nulo encontrado nas variáveis alvo")
else:
    print("❗ Atenção: valores nulos encontrados nas variáveis alvo")

## 5. Pipeline de Limpeza de Dados

Implementando pipeline sistemático de limpeza de dados seguindo as melhores práticas para análise de readmissão hospitalar.

In [ ]:
print("="*80)
print("PIPELINE DE LIMPEZA DOS DADOS")
print("="*80)

# Fazer uma cópia dos dados para limpeza
df_clean = df_raw.copy()
print(f"\n📊 Dados iniciais: {df_clean.shape}")

# ETAPA 1: Remover pacientes expirados (falecidos)
print("\n🔧 ETAPA 1: Removendo pacientes expirados")
print("Códigos de discharge_disposition_id para 'Expired': 11, 19, 20, 21")

expired_codes = [11, 19, 20, 21]
expired_count = df_clean['discharge_disposition_id'].isin(expired_codes).sum()
print(f"Registros com pacientes expirados: {expired_count}")

df_clean = df_clean[~df_clean['discharge_disposition_id'].isin(expired_codes)].copy()
print(f"Registros após remoção: {df_clean.shape[0]}")
print(f"Registros removidos: {len(df_raw) - len(df_clean)}")

# ETAPA 2: Tratar dados faltantes
print("\n🔧 ETAPA 2: Tratando dados faltantes")

# Verificar colunas com alto percentual de dados faltantes
columns_to_analyze = ['weight', 'payer_code', 'medical_specialty']
missing_analysis = {}

for col in columns_to_analyze:
    if col in df_clean.columns:
        missing_count = (df_clean[col] == '?').sum()
        missing_pct = (missing_count / len(df_clean)) * 100
        missing_analysis[col] = {'count': missing_count, 'percentage': missing_pct}
        print(f"Dados faltantes em '{col}': {missing_count} ({missing_pct:.1f}%)")

# Remover colunas com muitos dados faltantes (weight e payer_code)
columns_to_drop = ['weight', 'payer_code']
print(f"\nRemoção de colunas com muitos dados faltantes: {columns_to_drop}")
df_clean = df_clean.drop(columns_to_drop, axis=1)

# Tratar medical_specialty: substituir '?' por 'missing'
if 'medical_specialty' in df_clean.columns:
    missing_specialty_before = (df_clean['medical_specialty'] == '?').sum()
    df_clean['medical_specialty'] = df_clean['medical_specialty'].replace('?', 'missing')
    missing_specialty_after = (df_clean['medical_specialty'] == 'missing').sum()
    print(f"Valores '?' em medical_specialty substituídos por 'missing': {missing_specialty_after}")

# ETAPA 3: Remover duplicatas de pacientes (manter apenas primeira internação)
print("\n🔧 ETAPA 3: Removendo dados duplicados de pacientes")
print(f"Registros antes da remoção de duplicatas: {len(df_clean)}")
print(f"Pacientes únicos: {df_clean['patient_nbr'].nunique()}")

# Manter apenas a primeira internação de cada paciente
df_clean = df_clean.drop_duplicates(subset=['patient_nbr'], keep='first')

print(f"Registros após remoção de duplicatas: {len(df_clean)}")
print(f"Pacientes únicos após limpeza: {df_clean['patient_nbr'].nunique()}")
print(f"Registros de duplicatas removidos: {len(df_raw) - len(df_clean)}")

# ETAPA 4: Verificar distribuição da variável alvo após limpeza
print("\n🔧 ETAPA 4: Verificando distribuição da variável alvo após limpeza")
print("Distribuição da variável 'readmitted' após limpeza:")
print(df_clean['readmitted'].value_counts())

print(f"\nDistribuição da variável 'target' após limpeza:")
print(df_clean['target'].value_counts())
print(f"Proporção de readmissões em <30 dias após limpeza: {df_clean['target'].mean():.3f}")

# Resumo final da limpeza
print("\n" + "="*80)
print("RESUMO DA LIMPEZA DOS DADOS")
print("="*80)
print(f"📊 Registros originais: {len(df_raw):,}")
print(f"📊 Registros após limpeza: {len(df_clean):,}")
print(f"📊 Registros removidos: {len(df_raw) - len(df_clean):,}")
print(f"📊 Percentual mantido: {(len(df_clean) / len(df_raw)) * 100:.1f}%")
print(f"📊 Colunas originais: {df_raw.shape[1]}")
print(f"📊 Colunas após limpeza: {df_clean.shape[1]}")
print(f"📊 Dimensões finais: {df_clean.shape}")

# Visualizar impacto da limpeza na variável alvo
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Antes da limpeza
target_before = df_raw['target'].value_counts()
ax1.pie(target_before.values, labels=['Não Readmitido <30d', 'Readmitido <30d'], 
        autopct='%1.1f%%', colors=['skyblue', 'lightcoral'])
ax1.set_title(f'Distribuição da Variável Alvo\nAntes da Limpeza (n={len(df_raw):,})')

# Após a limpeza
target_after = df_clean['target'].value_counts()
ax2.pie(target_after.values, labels=['Não Readmitido <30d', 'Readmitido <30d'], 
        autopct='%1.1f%%', colors=['skyblue', 'lightcoral'])
ax2.set_title(f'Distribuição da Variável Alvo\nApós a Limpeza (n={len(df_clean):,})')

plt.tight_layout()
plt.show()

print("✅ Limpeza dos dados concluída com sucesso!")

## 6. Engenharia de Atributos e Codificação

Preparando as features para algoritmos de Machine Learning através de encoding de variáveis categóricas e engenharia de features.

In [ ]:
print("="*80)
print("ENGENHARIA DE FEATURES E ENCODING")
print("="*80)

# Preparar dados para modelagem
df_processed = df_clean.copy()

# ETAPA 1: Remover colunas desnecessárias para modelagem
print("\n🔧 ETAPA 1: Removendo colunas desnecessárias para modelagem")
cols_to_drop = ['encounter_id', 'patient_nbr', 'readmitted', 'diag_1', 'diag_2', 'diag_3']
print(f"Colunas a serem removidas: {cols_to_drop}")

# Verificar quais colunas existem antes de remover
existing_cols_to_drop = [col for col in cols_to_drop if col in df_processed.columns]
print(f"Colunas existentes que serão removidas: {existing_cols_to_drop}")

df_processed = df_processed.drop(existing_cols_to_drop, axis=1)
print(f"Dimensões após remoção: {df_processed.shape}")

# ETAPA 2: Identificar tipos de colunas
print("\n🔧 ETAPA 2: Identificando tipos de colunas")
numeric_cols = df_processed.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df_processed.select_dtypes(include=['object']).columns.tolist()

# Remover target da lista de features numéricas
if 'target' in numeric_cols:
    numeric_cols.remove('target')

print(f"📊 Colunas numéricas ({len(numeric_cols)}): {numeric_cols[:10]}{'...' if len(numeric_cols) > 10 else ''}")
print(f"📊 Colunas categóricas ({len(categorical_cols)}): {categorical_cols}")

# Analisar colunas categóricas em detalhes
print(f"\n📋 Análise detalhada das colunas categóricas:")
for col in categorical_cols:
    unique_values = df_processed[col].nunique()
    sample_values = df_processed[col].unique()[:5]
    print(f"  {col}: {unique_values} valores únicos, exemplos: {sample_values}")

# ETAPA 3: Aplicar One-Hot Encoding nas colunas categóricas
print(f"\n🔧 ETAPA 3: Aplicando One-Hot Encoding")
print(f"Aplicando encoding em {len(categorical_cols)} colunas categóricas")
print(f"Dimensões antes do encoding: {df_processed.shape}")

df_encoded = pd.get_dummies(df_processed, columns=categorical_cols, drop_first=True)

print(f"Dimensões após encoding: {df_encoded.shape}")
print(f"Novas features criadas: {df_encoded.shape[1] - df_processed.shape[1]}")

# ETAPA 4: Separar features (X) e variável alvo (y)
print(f"\n🔧 ETAPA 4: Separando features (X) e variável alvo (y)")

X = df_encoded.drop('target', axis=1)
y = df_encoded['target']

print(f"📊 Formato de X (features): {X.shape}")
print(f"📊 Formato de y (target): {y.shape}")
print(f"📊 Número total de features: {X.shape[1]}")

# ETAPA 5: Análise das features criadas
print(f"\n🔧 ETAPA 5: Análise das features criadas")

# Categorizar features por tipo
medication_cols = [col for col in X.columns if any(med in col.lower() for med in 
                  ['metformin', 'insulin', 'glyburide', 'glipizide', 'glimepiride', 'pioglitazone', 
                   'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 
                   'examide', 'citoglipton', 'tolbutamide', 'repaglinide', 'nateglinide', 
                   'chlorpropamide', 'acetohexamide'])]

demographic_cols = [col for col in X.columns if any(demo in col.lower() for demo in 
                   ['race', 'gender', 'age'])]

admission_cols = [col for col in X.columns if any(adm in col.lower() for adm in 
                 ['admission_type', 'discharge_disposition', 'admission_source'])]

lab_cols = [col for col in X.columns if any(lab in col.lower() for lab in 
           ['num_lab_procedures', 'num_procedures', 'num_medications', 'time_in_hospital'])]

print(f"📊 Features de medicamentos: {len(medication_cols)}")
print(f"📊 Features demográficas: {len(demographic_cols)}")
print(f"📊 Features de admissão: {len(admission_cols)}")
print(f"📊 Features laboratoriais/clínicas: {len(lab_cols)}")
print(f"📊 Outras features: {X.shape[1] - len(medication_cols) - len(demographic_cols) - len(admission_cols) - len(lab_cols)}")

# Verificar estatísticas básicas das features numéricas originais
print(f"\n📈 Estatísticas das features numéricas principais:")
numeric_features_summary = X[numeric_cols].describe()
display(numeric_features_summary)

# Visualizar distribuição de algumas features importantes
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Time in hospital
axes[0,0].hist(X['time_in_hospital'], bins=20, alpha=0.7, color='skyblue')
axes[0,0].set_title('Distribuição: Tempo de Internação')
axes[0,0].set_xlabel('Dias')

# Number of lab procedures
axes[0,1].hist(X['num_lab_procedures'], bins=20, alpha=0.7, color='lightgreen')
axes[0,1].set_title('Distribuição: Número de Procedimentos Laboratoriais')
axes[0,1].set_xlabel('Número de Procedimentos')

# Number of medications
axes[1,0].hist(X['num_medications'], bins=20, alpha=0.7, color='lightcoral')
axes[1,0].set_title('Distribuição: Número de Medicamentos')
axes[1,0].set_xlabel('Número de Medicamentos')

# Number of diagnoses
axes[1,1].hist(X['number_diagnoses'], bins=20, alpha=0.7, color='lightyellow')
axes[1,1].set_title('Distribuição: Número de Diagnósticos')
axes[1,1].set_xlabel('Número de Diagnósticos')

plt.tight_layout()
plt.show()

print("✅ Engenharia de features concluída com sucesso!")
print(f"✅ Dataset pronto para Machine Learning: {X.shape[0]} amostras, {X.shape[1]} features")

## 7. Divisão dos Dados para Treinamento do Modelo

Dividindo os dados em conjuntos de treino e teste usando amostragem estratificada para manter o equilíbrio das classes.

In [ ]:
print("="*80)
print("DIVISÃO DOS DADOS PARA TREINAMENTO")
print("="*80)

# Configurações da divisão
test_size = 0.2
random_state = 42

print(f"\n⚙️ Configurações da divisão:")
print(f"Proporção de teste: {test_size} ({test_size*100}%)")
print(f"Proporção de treino: {1-test_size} ({(1-test_size)*100}%)")
print(f"Random state: {random_state} (para reprodutibilidade)")
print(f"Estratificação: Sim (mantém proporção de classes)")

# Verificar distribuição antes da divisão
print(f"\n📊 Distribuição da variável alvo antes da divisão:")
target_distribution = y.value_counts()
target_distribution_pct = y.value_counts(normalize=True) * 100
print(f"Classe 0 (Não readmitido <30d): {target_distribution[0]} ({target_distribution_pct[0]:.1f}%)")
print(f"Classe 1 (Readmitido <30d): {target_distribution[1]} ({target_distribution_pct[1]:.1f}%)")

# Dividir os dados
print(f"\n🔧 Executando divisão dos dados...")
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=test_size,
    random_state=random_state,
    stratify=y  # MUITO importante para datasets desbalanceados
)

print(f"✅ Divisão concluída com sucesso!")

# Verificar tamanhos dos conjuntos
print(f"\n📊 Tamanhos dos conjuntos criados:")
print(f"X_train: {X_train.shape} (amostras: {X_train.shape[0]}, features: {X_train.shape[1]})")
print(f"X_test:  {X_test.shape} (amostras: {X_test.shape[0]}, features: {X_test.shape[1]})")
print(f"y_train: {y_train.shape} (amostras: {y_train.shape[0]})")
print(f"y_test:  {y_test.shape} (amostras: {y_test.shape[0]})")

# Verificar distribuição das classes nos conjuntos
print(f"\n📊 Distribuição das classes nos conjuntos:")

# Conjunto de treino
train_distribution = y_train.value_counts()
train_distribution_pct = y_train.value_counts(normalize=True) * 100
print(f"\nConjunto de TREINO:")
print(f"  Classe 0: {train_distribution[0]} ({train_distribution_pct[0]:.1f}%)")
print(f"  Classe 1: {train_distribution[1]} ({train_distribution_pct[1]:.1f}%)")

# Conjunto de teste
test_distribution = y_test.value_counts()
test_distribution_pct = y_test.value_counts(normalize=True) * 100
print(f"\nConjunto de TESTE:")
print(f"  Classe 0: {test_distribution[0]} ({test_distribution_pct[0]:.1f}%)")
print(f"  Classe 1: {test_distribution[1]} ({test_distribution_pct[1]:.1f}%)")

# Visualizar distribuição dos conjuntos
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# Dataset completo
y.value_counts().plot(kind='bar', ax=ax1, color=['skyblue', 'lightcoral'])
ax1.set_title(f'Dataset Completo\n(n={len(y):,})')
ax1.set_xlabel('Classe')
ax1.set_ylabel('Número de Amostras')
ax1.set_xticklabels(['Não Readmitido <30d', 'Readmitido <30d'], rotation=45)

# Conjunto de treino
y_train.value_counts().plot(kind='bar', ax=ax2, color=['skyblue', 'lightcoral'])
ax2.set_title(f'Conjunto de Treino\n(n={len(y_train):,})')
ax2.set_xlabel('Classe')
ax2.set_ylabel('Número de Amostras')
ax2.set_xticklabels(['Não Readmitido <30d', 'Readmitido <30d'], rotation=45)

# Conjunto de teste
y_test.value_counts().plot(kind='bar', ax=ax3, color=['skyblue', 'lightcoral'])
ax3.set_title(f'Conjunto de Teste\n(n={len(y_test):,})')
ax3.set_xlabel('Classe')
ax3.set_ylabel('Número de Amostras')
ax3.set_xticklabels(['Não Readmitido <30d', 'Readmitido <30d'], rotation=45)

plt.tight_layout()
plt.show()

# Verificar se a estratificação funcionou
print(f"\n✅ Verificação da estratificação:")
original_ratio = target_distribution_pct[1]
train_ratio = train_distribution_pct[1]
test_ratio = test_distribution_pct[1]

print(f"Taxa de readmissão original: {original_ratio:.1f}%")
print(f"Taxa de readmissão no treino: {train_ratio:.1f}%")
print(f"Taxa de readmissão no teste: {test_ratio:.1f}%")

# Verificar se as diferenças são pequenas (boa estratificação)
diff_train = abs(original_ratio - train_ratio)
diff_test = abs(original_ratio - test_ratio)

if diff_train < 1.0 and diff_test < 1.0:
    print("✅ Estratificação bem-sucedida: distribuições similares entre conjuntos")
else:
    print("⚠️ Atenção: diferenças na distribuição podem afetar a avaliação")

print(f"\n🎯 DADOS PRONTOS PARA MACHINE LEARNING:")
print(f"📊 Conjunto de treino: {X_train.shape[0]:,} amostras, {X_train.shape[1]} features")
print(f"📊 Conjunto de teste: {X_test.shape[0]:,} amostras, {X_test.shape[1]} features")
print(f"📊 Taxa de readmissão balanceada entre conjuntos")
print(f"📊 Dados prontos para treinamento de modelos!")

## 8. Exportação dos Dados Processados

Salvando os datasets limpos e processados para uso posterior em modelagem e análise.

In [ ]:
print("="*80)
print("EXPORTANDO DADOS PROCESSADOS")
print("="*80)

# Definir caminhos de saída
output_dir = '../data/'
clean_data_path = os.path.join(output_dir, 'diabetic_data_clean.csv')
processed_data_path = os.path.join(output_dir, 'diabetic_data_processed.csv')

# Caminhos para conjuntos de treino e teste
train_features_path = os.path.join(output_dir, 'X_train.csv')
test_features_path = os.path.join(output_dir, 'X_test.csv')
train_target_path = os.path.join(output_dir, 'y_train.csv')
test_target_path = os.path.join(output_dir, 'y_test.csv')

print(f"\n💾 Salvando datasets...")

# 1. Salvar dados limpos (intermediário)
df_clean.to_csv(clean_data_path, index=False)
print(f"✅ Dados limpos salvos: {clean_data_path}")
print(f"   Dimensões: {df_clean.shape}")

# 2. Salvar dados processados completos (com encoding)
df_encoded.to_csv(processed_data_path, index=False)
print(f"✅ Dados processados salvos: {processed_data_path}")
print(f"   Dimensões: {df_encoded.shape}")

# 3. Salvar conjuntos de treino e teste separadamente
X_train.to_csv(train_features_path, index=False)
X_test.to_csv(test_features_path, index=False)
y_train.to_csv(train_target_path, index=False, header=['target'])
y_test.to_csv(test_target_path, index=False, header=['target'])

print(f"✅ Conjuntos de Machine Learning salvos:")
print(f"   Features de treino: {train_features_path} {X_train.shape}")
print(f"   Features de teste: {test_features_path} {X_test.shape}")
print(f"   Target de treino: {train_target_path} {y_train.shape}")
print(f"   Target de teste: {test_target_path} {y_test.shape}")

# 4. Criar arquivo de metadados com informações do processamento
metadata = {
    'pipeline_info': {
        'data_original_shape': df_raw.shape,
        'data_clean_shape': df_clean.shape,
        'data_processed_shape': df_encoded.shape,
        'train_shape': X_train.shape,
        'test_shape': X_test.shape,
        'target_distribution': {
            'total_readmitted_30d': int(y.sum()),
            'total_not_readmitted_30d': int(len(y) - y.sum()),
            'train_readmitted_30d': int(y_train.sum()),
            'test_readmitted_30d': int(y_test.sum()),
            'readmission_rate': float(y.mean())
        }
    },
    'processing_steps': {
        'expired_patients_removed': int(len(df_raw) - len(df_clean)),
        'columns_dropped': existing_cols_to_drop,
        'missing_data_handled': ['weight', 'payer_code', 'medical_specialty'],
        'duplicates_removed': 'kept_first_encounter_per_patient',
        'encoding_applied': 'one_hot_encoding_categorical_vars',
        'total_features_created': int(X.shape[1])
    },
    'data_quality': {
        'missing_values_remaining': int(df_encoded.isnull().sum().sum()),
        'categorical_encoded': len(categorical_cols),
        'numeric_features': len(numeric_cols),
        'total_features': int(X.shape[1])
    }
}

# Salvar metadados
import json
metadata_path = os.path.join(output_dir, 'processing_metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✅ Metadados salvos: {metadata_path}")

# 5. Criar resumo executivo do processamento
summary_path = os.path.join(output_dir, 'data_processing_summary.txt')
with open(summary_path, 'w', encoding='utf-8') as f:
    f.write("RESUMO DO PROCESSAMENTO DE DADOS - READMISSÃO HOSPITALAR DIABÉTICA\\n")
    f.write("="*70 + "\\n\\n")
    
    f.write(f"Data de processamento: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\\n\\n")
    
    f.write("TRANSFORMAÇÕES APLICADAS:\\n")
    f.write(f"1. Dados originais: {df_raw.shape[0]:,} registros, {df_raw.shape[1]} colunas\\n")
    f.write(f"2. Remoção de pacientes expirados: -{len(df_raw) - len(df_clean):,} registros\\n")
    f.write(f"3. Tratamento de dados faltantes: removidas colunas weight, payer_code\\n")
    f.write(f"4. Remoção de duplicatas por paciente: mantida primeira internação\\n")
    f.write(f"5. Dados limpos: {df_clean.shape[0]:,} registros, {df_clean.shape[1]} colunas\\n")
    f.write(f"6. Engenharia de features: aplicado one-hot encoding\\n")
    f.write(f"7. Dados processados: {df_encoded.shape[0]:,} registros, {df_encoded.shape[1]} colunas\\n")
    f.write(f"8. Divisão treino/teste: {len(X_train):,}/{len(X_test):,} (80%/20%)\\n\\n")
    
    f.write("DISTRIBUIÇÃO DA VARIÁVEL ALVO:\\n")
    f.write(f"- Readmitidos em <30 dias: {y.sum():,} ({y.mean()*100:.1f}%)\\n")
    f.write(f"- Não readmitidos em <30 dias: {len(y) - y.sum():,} ({(1-y.mean())*100:.1f}%)\\n\\n")
    
    f.write("ARQUIVOS GERADOS:\\n")
    f.write(f"- diabetic_data_clean.csv: Dados limpos intermediários\\n")
    f.write(f"- diabetic_data_processed.csv: Dados completos processados\\n")
    f.write(f"- X_train.csv, y_train.csv: Conjunto de treinamento\\n")
    f.write(f"- X_test.csv, y_test.csv: Conjunto de teste\\n")
    f.write(f"- processing_metadata.json: Metadados detalhados\\n")

print(f"✅ Resumo executivo salvo: {summary_path}")

# Visualização final do pipeline
print(f"\\n" + "="*80)
print("PIPELINE DE PROCESSAMENTO CONCLUÍDO")
print("="*80)

pipeline_summary = pd.DataFrame({
    'Etapa': ['Dados Originais', 'Após Limpeza', 'Após Encoding', 'Treino', 'Teste'],
    'Registros': [df_raw.shape[0], df_clean.shape[0], df_encoded.shape[0], X_train.shape[0], X_test.shape[0]],
    'Colunas/Features': [df_raw.shape[1], df_clean.shape[1], df_encoded.shape[1], X_train.shape[1], X_test.shape[1]]
})

display(pipeline_summary)

print(f"\\n🎯 PRÓXIMOS PASSOS:")
print(f"1. ✅ Dados preparados e prontos para Machine Learning")
print(f"2. 🤖 Implementar modelos de classificação (Random Forest, SVM, etc.)")
print(f"3. 📊 Avaliar performance dos modelos (accuracy, precision, recall, F1)")
print(f"4. 🔧 Fazer tuning de hiperparâmetros")
print(f"5. 📈 Interpretar resultados e features mais importantes")

print(f"\\n✅ Todos os dados foram processados e salvos com sucesso!")
print(f"💾 Localização dos arquivos: {os.path.abspath(output_dir)}")

## 5. Análise dos Mapeamentos de IDs

Esta seção analisa os dados enriquecidos com descrições legíveis dos códigos IDs, melhorando a interpretabilidade dos dados.

In [ ]:
# Importar o utilitário de mapeamentos
from src.id_mapping_utils import IDMappingUtils

# Carregar mapeamentos
print("Carregando mapeamentos de IDs...")
id_mapper = IDMappingUtils()
mappings = id_mapper.load_mappings()

# Exibir resumo dos mapeamentos
print("\n" + "="*60)
print("RESUMO DOS MAPEAMENTOS DISPONÍVEIS")
print("="*60)

summary = id_mapper.get_mapping_summary()
for mapping_type, info in summary.items():
    print(f"\n{mapping_type.upper()}:")
    print(f"  Número de itens: {info['count']}")
    print(f"  Range de IDs: {info['id_range']}")
    print(f"  Exemplos: {info['sample_values']}")

# Validar mapeamentos com os dados atuais
print("\n" + "="*60)
print("VALIDAÇÃO DOS MAPEAMENTOS COM OS DADOS")
print("="*60)

validation_report = id_mapper.validate_mappings(df_clean)
for mapping_type, report in validation_report.items():
    coverage = report['coverage_rate'] * 100
    print(f"\n{mapping_type}:")
    print(f"  IDs únicos no dataset: {report['total_unique_ids']}")
    print(f"  IDs com mapeamento: {report['mapped_ids']}")
    print(f"  Taxa de cobertura: {coverage:.1f}%")
    if report['unmapped_ids']:
        print(f"  IDs não mapeados: {report['unmapped_ids']}")

In [ ]:
# Aplicar mapeamentos aos dados limpos
print("Aplicando mapeamentos aos dados...")
df_enriched = id_mapper.apply_mappings_to_dataframe(df_clean)

print(f"\nDimensões antes dos mapeamentos: {df_clean.shape}")
print(f"Dimensões após mapeamentos: {df_enriched.shape}")

# Verificar novas colunas criadas
desc_columns = [col for col in df_enriched.columns if col.endswith('_desc')]
print(f"\nNovas colunas descritivas criadas: {desc_columns}")

# Exibir amostra dos dados enriquecidos
print("\n" + "="*60)
print("AMOSTRA DOS DADOS ENRIQUECIDOS")
print("="*60)

# Mostrar comparação entre IDs e descrições
for col in desc_columns:
    id_col = col.replace('_desc', '_id')
    if id_col in df_enriched.columns:
        print(f"\n{col.upper()}:")
        comparison = df_enriched[[id_col, col]].drop_duplicates().sort_values(id_col)
        print(comparison.head(10))

# Análise estatística das categorias com descrições
print("\n" + "="*60)
print("ANÁLISE ESTATÍSTICA COM DESCRIÇÕES")
print("="*60)

for col in desc_columns:
    print(f"\nDistribuição de {col}:")
    distribution = df_enriched[col].value_counts()
    print(distribution.head(10))
    
    # Taxa de readmissão por categoria
    if 'target' in df_enriched.columns:
        print(f"\nTaxa de readmissão por {col}:")
        readmission_rate = df_enriched.groupby(col)['target'].agg(['count', 'mean']).round(3)
        readmission_rate.columns = ['Total_Casos', 'Taxa_Readmissao']
        readmission_rate = readmission_rate.sort_values('Taxa_Readmissao', ascending=False)
        print(readmission_rate.head(10))

In [ ]:
# Criar visualizações dos dados mapeados
print("Criando visualizações dos dados enriquecidos...")

# Configurar o estilo dos gráficos
plt.style.use('seaborn-v0_8')
fig = plt.figure(figsize=(20, 15))

# 1. Distribuição dos Tipos de Admissão
if 'admission_type_desc' in df_enriched.columns:
    plt.subplot(3, 2, 1)
    admission_counts = df_enriched['admission_type_desc'].value_counts()
    plt.pie(admission_counts.values, labels=admission_counts.index, autopct='%1.1f%%')
    plt.title('Distribuição dos Tipos de Admissão', fontsize=14, fontweight='bold')

# 2. Taxa de Readmissão por Tipo de Admissão
if 'admission_type_desc' in df_enriched.columns:
    plt.subplot(3, 2, 2)
    admission_readmission = df_enriched.groupby('admission_type_desc')['target'].mean().sort_values(ascending=False)
    bars = plt.bar(range(len(admission_readmission)), admission_readmission.values)
    plt.xticks(range(len(admission_readmission)), admission_readmission.index, rotation=45, ha='right')
    plt.ylabel('Taxa de Readmissão')
    plt.title('Taxa de Readmissão por Tipo de Admissão', fontsize=14, fontweight='bold')
    
    # Adicionar valores nas barras
    for i, v in enumerate(admission_readmission.values):
        plt.text(i, v + 0.005, f'{v:.3f}', ha='center', va='bottom')

# 3. Top 10 Disposições de Alta
if 'discharge_disposition_desc' in df_enriched.columns:
    plt.subplot(3, 2, 3)
    top_discharge = df_enriched['discharge_disposition_desc'].value_counts().head(10)
    plt.barh(range(len(top_discharge)), top_discharge.values)
    plt.yticks(range(len(top_discharge)), [label[:30] + '...' if len(label) > 30 else label 
                                          for label in top_discharge.index])
    plt.xlabel('Número de Casos')
    plt.title('Top 10 Disposições de Alta', fontsize=14, fontweight='bold')

# 4. Taxa de Readmissão por Fonte de Admissão
if 'admission_source_desc' in df_enriched.columns:
    plt.subplot(3, 2, 4)
    source_readmission = df_enriched.groupby('admission_source_desc')['target'].agg(['count', 'mean'])
    # Filtrar apenas fontes com pelo menos 100 casos
    source_readmission = source_readmission[source_readmission['count'] >= 100]
    source_readmission = source_readmission.sort_values('mean', ascending=False)
    
    bars = plt.bar(range(len(source_readmission)), source_readmission['mean'].values)
    plt.xticks(range(len(source_readmission)), 
               [label[:15] + '...' if len(label) > 15 else label 
                for label in source_readmission.index], 
               rotation=45, ha='right')
    plt.ylabel('Taxa de Readmissão')
    plt.title('Taxa de Readmissão por Fonte de Admissão (≥100 casos)', fontsize=14, fontweight='bold')

# 5. Heatmap de correlação entre variáveis categóricas mapeadas
if len(desc_columns) >= 2:
    plt.subplot(3, 2, 5)
    # Criar matriz de correlação para variáveis categóricas
    from sklearn.preprocessing import LabelEncoder
    
    encoded_data = pd.DataFrame()
    for col in desc_columns[:3]:  # Usar apenas as 3 primeiras para evitar sobrecarga
        if col in df_enriched.columns:
            le = LabelEncoder()
            encoded_data[col] = le.fit_transform(df_enriched[col].fillna('Missing'))
    
    if not encoded_data.empty:
        correlation_matrix = encoded_data.corr()
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
        plt.title('Correlação entre Variáveis Categóricas Mapeadas', fontsize=14, fontweight='bold')

# 6. Distribuição de Readmissão por combinação de categorias
if 'admission_type_desc' in df_enriched.columns and 'discharge_disposition_desc' in df_enriched.columns:
    plt.subplot(3, 2, 6)
    # Criar crosstab das duas principais categorias
    cross_tab = pd.crosstab(df_enriched['admission_type_desc'], 
                           df_enriched['discharge_disposition_desc'], 
                           normalize='index')
    # Mostrar apenas as 5 disposições mais comuns
    top_dispositions = df_enriched['discharge_disposition_desc'].value_counts().head(5).index
    cross_tab_filtered = cross_tab[top_dispositions]
    
    sns.heatmap(cross_tab_filtered, annot=True, fmt='.2f', cmap='Blues')
    plt.title('Proporção de Disposições de Alta por Tipo de Admissão', fontsize=14, fontweight='bold')
    plt.xlabel('Disposição de Alta')
    plt.ylabel('Tipo de Admissão')

plt.tight_layout()
plt.show()

# Estatísticas resumidas dos mapeamentos
print("\n" + "="*60)
print("ESTATÍSTICAS RESUMIDAS DOS MAPEAMENTOS")
print("="*60)

for col in desc_columns:
    print(f"\n{col.upper()}:")
    print(f"  Categorias únicas: {df_enriched[col].nunique()}")
    print(f"  Categoria mais comum: {df_enriched[col].mode().iloc[0]}")
    print(f"  Valores nulos: {df_enriched[col].isnull().sum()}")
    
    if 'target' in df_enriched.columns:
        most_risky = df_enriched.groupby(col)['target'].mean().sort_values(ascending=False).iloc[0]
        most_risky_category = df_enriched.groupby(col)['target'].mean().sort_values(ascending=False).index[0]
        print(f"  Categoria com maior risco: {most_risky_category} ({most_risky:.3f})")

print(f"\nDados enriquecidos salvos em memória com {df_enriched.shape[0]} registros e {df_enriched.shape[1]} colunas.")